In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

#### Končni rezultat oz. izpis uporabniku
    - Ko model prepozna dan/noč ali jasno/deževno/megleno želim uporabniku olajšati vožnjo zato bo glede na te spremeljivke se izvedla ustrezna funkcija kot je: vklop meglenk, brisalcev, ambientne osvetljave, dvig senčnikov
    - Po vhodu slike mora model prepoznati 1

In [2]:
data_dir = "Podatki_split/weather"

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [5]:
batch_size = 32

train_dataset = datasets.ImageFolder(root=f'{data_dir}/train', transform=transform)
val_dataset   = datasets.ImageFolder(root=f'{data_dir}/val', transform=transform)
test_dataset  = datasets.ImageFolder(root=f'{data_dir}/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [6]:
print("Oznake/Labels:", train_dataset.classes)
print("Število slik v učnem sklopu:", len(train_dataset))

Oznake/Labels: ['clear', 'foggy', 'rainy']
Število slik v učnem sklopu: 573


#### Training a Classifier: https://docs.pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
##### Docs: https://www.geeksforgeeks.org/introduction-convolution-neural-network/

In [7]:
import torch.nn as nn
import torch.nn.functional as F

In [8]:
class WeatherCNN(nn.Module):
    def __init__(self):
        super(WeatherCNN, self).__init__()

        # Dve konvolucijske plasti
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)

        # Popolnoma povezana/Fully connected plast
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 3)  # Izhod zdaj 3 oznake, pri TimeOfDay 2

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
#Inicializacija modela in nastavitev loss funkcije in optimizerja
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_weather = WeatherCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_weather.parameters(), lr=0.001)

In [11]:
def validate(model, validation_loader, criterion):
    model.eval()
    
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = running_loss / len(validation_loader)
    accuracy = 100 * correct / total

    return avg_loss, accuracy

In [ ]:
from tqdm import tqdm

num_epochs = 5 # Ni isto kot epizoda!, epoch - SL, episode - RL

for epoch in range(num_epochs):
    running_loss = 0.0
    model_weather.train()

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()                  # Tukaj se izvede backpropagation
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix({"Loss": running_loss / (progress_bar.n + 1)})

    avg_loss = running_loss / len(train_loader)

    # Validacija
    val_loss, val_accuracy = validate(model, val_loader, criterion)
    
    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {avg_loss:.4f}, "
          f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")